Mit Colab kann **dieses Jupyter-Notebook interaktiv im Browser** gestartet werden:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DonGoginho/myPy/blob/main/parlamentsdienste_ris_api/pd_ris_api_al_geschaefte.ipynb)

Und hier ein Jupyter-Notebook zur [generellen Verwendung der RIS-API](https://colab.research.google.com/github/opendatazurich/opendatazurich.github.io/blob/master/ris-api/RIS-API-Beispiele.ipynb)
# Python-Abfrage an RIS-API nach Geschäftern, die von der Alternativen Liste eingegeben wurden

Das Datenmodell der [**RIS-API ist als Swagger File**](https://opendatazurich.github.io/ris-api/docs/) verfügbar. Für das Verständnis ist dieses daher wichtig. 

**Inhaltsverzeichnis:**

1. [Mitglieder der Alternativen Liste](#Mitglieder-der-AL)
1. [Gemeinderats-Geschäfte](#Gemeinderats-Geschäfte)

In [78]:
%pip install requests pandas

In [79]:
import requests
from pprint import pprint
import pandas as pd
import random

In [80]:
SSL_VERIFY = True
# evtl. SSL_VERIFY auf False setzen wenn die Verbindung zu https://www.gemeinderat-zuerich.ch nicht klappt (z.B. wegen Proxy)
# Um die SSL Verifikation auszustellen, bitte die nächste Zeile einkommentieren ("#" entfernen)
# SSL_VERIFY = False

In [81]:
if not SSL_VERIFY:
    import urllib3
    urllib3.disable_warnings()

## Infos zu AL
- 'parteiId': 'df5ccd54-009e-40ae-b687-4ce88a71f5d8'
- 'fraktionsId': '53bf7290-4055-455b-bb1f-2ba64e76294d'

## Mitglieder der AL

Um alle Mitglieder einer Partei zu bekommen, müssen folgende Abfragen gemacht werden:
    
    1. ID der Partei holen
    1. Suche nach Mitgliedern der Partei (mit dieser ID)
    1. *Optional:* Detail-Informationen eines Mitglieds holen
    
    

### Alle relevanten Mitglieder-Parameter holen

Lese alle Parameter zu den Mitgliedern in `params` ein

In [82]:
headers = {'Accept': 'application/json'}
r = requests.get('https://www.gemeinderat-zuerich.ch/api/Mitglieder/parameter', headers=headers, verify=SSL_VERIFY)
params = r.json()
#params

Lese die Informationen zu den Parteien aus `params`heraus

In [83]:
# get id of AL
parties = params['Parteien']
#party = {'parteiId': 'df5ccd54-009e-40ae-b687-4ce88a71f5d8'}
parties

[{'Id': 'df5ccd54-009e-40ae-b687-4ce88a71f5d8', 'Name': 'AL'},
 {'Id': '5964b653-a32a-4e0c-92a1-07720566a92c', 'Name': 'CVP'},
 {'Id': '187b7ad1-2095-43cc-b257-e9834a54a961', 'Name': 'EVP'},
 {'Id': '8240033e-4818-4406-8f73-b7829d99ddee', 'Name': 'FDP'},
 {'Id': '0a8eb90f-6d09-47ac-99fe-ab97a77d4954', 'Name': 'GLP'},
 {'Id': 'fe956d66-e14d-4739-82cf-c12202d96445', 'Name': 'Grüne'},
 {'Id': '9e037de7-40a4-4fa3-85b7-f4c30cbb26c8', 'Name': 'Parteilos'},
 {'Id': 'c6fbb7c5-3aca-4797-b81e-209f27c4f3cc', 'Name': 'SD'},
 {'Id': '6cb1a3bb-8e26-4818-a8f5-0ea49ab3cd57', 'Name': 'SP'},
 {'Id': '8e484ad9-6e46-4d39-bc05-fbf18be11b0c', 'Name': 'SVP'}]

Für diese Anfrage ist die AL relevant. Definiere daher als `party`die gesuchte Partei

In [84]:
party = {'parteiId': 'df5ccd54-009e-40ae-b687-4ce88a71f5d8','Name': 'AL'}
party

{'Name': 'AL', 'parteiId': 'df5ccd54-009e-40ae-b687-4ce88a71f5d8'}

### Mitglieder der AL abfragen

In der Query wird die ParteiId angegeben, welche für die Abfrage des Endpoints relevant ist.
In `members`wird das Resultat der Abfrage als JSON-Array eingelesen.

In [85]:
query = {'parteiId':  'df5ccd54-009e-40ae-b687-4ce88a71f5d8'}
#query = {'fraktionsId':  '53bf7290-4055-455b-bb1f-2ba64e76294d'}
headers = {'Accept': 'application/json'}
r = requests.get('https://www.gemeinderat-zuerich.ch/api/Mitglieder/suchen', params=query, headers=headers, verify=SSL_VERIFY)
members = r.json()
#members

### Details zu den AL-Mitgliedern abfragen

Mit dem `/Mitglieder/details` Endpunkt können Details eines Mitglieds abgefragt werden. Dazu muss die Mitglieder-ID jedes Mitglieds herausgelesen und über die API abgefragt werden. Dies erfolgt über die Funktion `get_member_details()`.

In [105]:
#members

In [106]:
def get_member_details(member):
    #print(member)
    query = {'mid': member['Id']}
    #print(query)
    headers = {'Accept': 'application/json'}
    r = requests.get('https://www.gemeinderat-zuerich.ch/api/Mitglieder/details', params=query, headers=headers, verify=SSL_VERIFY)
    details = r.json()
    
    # trim "Wahlkreis" and "Wohnkreis" since they have a padding
    details['Wahlkreis'] = details['Wahlkreis'].strip()
    if details['Wohnkreis']:
        details['Wohnkreis'] = details['Wohnkreis'].strip()
    # add original member to keep WahlkreisOrderBy column
    details.update(member)
    return details
    
member_details = [get_member_details(m) for m in members]
df = pd.DataFrame(member_details)
df.head(3)

,Id,Name,Vorname,Anrede,Titel,Geburtstag,Wohnkreis,Beruf,Partei,Fraktion,Wahlkreis,Sitznummer,GruppenMitgliedschaften,Adressen,EmailPrivat,EmailGeschaeftlich,Mobiltelefon,MobiltelefonGeschaeftlich,TelefonGeschaeftlich,TelefonPrivat,Internetauftritt,Interessenverbindungen,NameInUrl,WahlkreisOrderBy
0,ee8204f4-9290-49ca-99ae-d18a22a0a513,Angst,Walter,Herr,None,1961-09-22T00:00:00,3,Leiter Kommunikation,AL,AL,3,None,"[{'Von': '2002-04-10T00:00:00', 'Bis': None, '...","[{'Adressart': 'Postadresse', 'Strasse1': None...",None,walterangst@gmail.com,079 288 56 92,None,044 296 90 30,None,None,"- Mieterinnen– und Mieterverband Zürich, Leite...",Walter%20Angst,101
1,bbfb8ca0-91b1-448d-b1fc-c5e3bb35c158,Eberle,Natalie,Frau,None,1967-04-24T00:00:00,9,Soziale Quartierentwicklung,AL,AL,3,None,"[{'Von': '2018-05-16T00:00:00', 'Bis': None, '...","[{'Adressart': 'Postadresse', 'Strasse1': None...",,quartiernetz@qnf.ch,,None,043 537 80 17,,None,"- Quartiernetz Friesenberg, Stellenleitung",Natalie%20Eberle,101
2,4a1be35e-3f69-48a4-85b1-3fee07b006ae,Garcia Nuñez,David,Herr,Dr.,1975-05-16T00:00:00,1,Arzt,AL,AL,4 und 5,None,"[{'Von': '2017-01-19T00:00:00', 'Bis': None, '...","[{'Adressart': 'Postadresse', 'Strasse1': '', ...",,david.garcia@al-zh.ch,,None,,,None,"- Volkshaus Zürich, Mitglied des Stiftungsrats...",David%20Garcia%20Nu%c3%b1ez,102


In [107]:
# Beispiel einer Grafik dazu
#count_by_wahlkreis = df[['Wahlkreis', 'WahlkreisOrderBy']].groupby(['Wahlkreis', 'WahlkreisOrderBy']).size().reset_index(name='Anzahl')
#count_by_wahlkreis = count_by_wahlkreis.sort_values(by=['WahlkreisOrderBy'])
#count_by_wahlkreis.plot.bar(x='Wahlkreis', y='Anzahl')

# Anfrage der Parlamentsdienste

Ziel dieser Anfrage ist es eine Liste zu erhalten, welche alle Geschäfte aufführt, welche seitens AL-Ratsmitglied oder der AL-Fraktion jemals eingereicht wurden. Dies sollten rund 660 Geschäfte sein.

Vorgegebene Bedingungen:
- Geschäftsart=Motion, Postulat, Interpellation, Schriftliche Anfrage, Beschlussantrag oder Globalbudgetantrag
- Akteure/Einreichende Person=AL (muss der Partei "AL" angehören)
- Zeitraum=unbegrenzt 

IDs zur AL
- 'parteiId': 'df5ccd54-009e-40ae-b687-4ce88a71f5d8'
- 'fraktionId': '53bf7290-4055-455b-bb1f-2ba64e76294d'

Resultat: 
Liste mit Angaben zu
- Geschäftsnummer
- Geschäftstitel
- Geschäftsart
- Einreichende Person/Partei/Fraktion: AL
- Link auf das Geschäft auf der Website des Gemeinderats



## Gemeinderats-Geschäfte die von der AL eingereicht wurden

Im der Resultatsliste sollen alls Geschäfte aufgelistet werden, welche die AL entweder als Partei oder als Fraktion eingegeben hat.

1.   Stelle zuerst den Request auf die Geschäfte, welche von der AL **als Fraktion** eingegeben wurden und übergebe das Resultat aus einem JSON (`result_geschaefte_fraktion`) in einen Dataframe (`df_deals_fraktion`).

In [109]:
#query = {'personId': member['Id']}

# via swagger: URL mit einem OR Operatoren drin
#https://www.gemeinderat-zuerich.ch/api/Geschaeft/suchen?parteiId=df5ccd54-009e-40ae-b687-4ce88a71f5d8|fraktionId=53bf7290-4055-455b-bb1f-2ba64e76294d&orderBy=geschaeftsart

query_fraktion = {'fraktionId': '53bf7290-4055-455b-bb1f-2ba64e76294d', 'activePage':1, 'pageSize':700}
headers = {'Accept': 'application/json'}
r = requests.get('https://www.gemeinderat-zuerich.ch/api/Geschaeft', 
                 params=query_fraktion, 
                 headers=headers, 
                 verify=SSL_VERIFY)
result_geschaefte_fraktion = r.json()
df_deals_fraktion = pd.DataFrame(result_geschaefte_fraktion['Geschaefte'])
print('done')




done


In [118]:
df_deals_fraktion.info

GeschaeftId         object
Geschaeftsjahr       int64
Geschaeftsnummer     int64
Geschaeftstitel     object
Geschaeftsart       object
dtype: object



2.   Stelle anschliessend den Request auf die Geschäfte, welche von der AL **als Partei** eingegeben wurden und übergebe das Resultat aus einem JSON (result_geschaefte_partei) in einen Dataframe (df_deals_partei).

In [154]:
query_partei = {'parteiId': 'df5ccd54-009e-40ae-b687-4ce88a71f5d8', 'activePage':1, 'pageSize':700}
headers = {'Accept': 'application/json'}
r = requests.get('https://www.gemeinderat-zuerich.ch/api/Geschaeft', 
                 params=query_partei, 
                 headers=headers, 
                 verify=SSL_VERIFY)
result_geschaefte_partei = r.json()
df_deals_partei = pd.DataFrame(result_geschaefte_partei['Geschaefte'])
print('done')


done


In [113]:
df_deals_partei.info

<bound method DataFrame.info of                               GeschaeftId  ...         Geschaeftsart
0    12f72f93-eef1-4dea-af7c-40dc4f7201c8  ...              Postulat
1    23e9191b-f4d1-4897-8380-074d3918f98e  ...              Postulat
2    513e2c8e-60ae-42af-a571-0c7a031922a6  ...  Schriftliche Anfrage
3    733659d9-f3ab-4513-b105-c264b5bd699b  ...  Schriftliche Anfrage
4    b0b223cb-a96d-465d-8a50-cb307a50291d  ...              Postulat
..                                    ...  ...                   ...
603  1afa9df6-8635-480d-a8b9-b8214b7c6613  ...                Motion
604  c928d093-0db6-4eaa-b30c-6196035b99e5  ...              Postulat
605  055190a3-536d-4dbe-97fb-a39677fc57d9  ...              Postulat
606  09ece0e7-4efc-4fcc-a825-847157382b42  ...              Postulat
607  94afa6b8-e776-455b-b2d9-bb34975c5f47  ...                Motion

[608 rows x 5 columns]>

Füge nun beide DataFrames der Teilanfragen (1 und 2) zusammen. 
`ignore_index` ist hier wichtig, sonst funktioniert das zusammenhängen nicht wie erwünscht.
Sortiere danach den DataFrame absteigend nach Geschäftsjahr und Geschäftsnummer und schreibe ihn in den DataFrame df_deals.


In [155]:
df_deals_appended = df_deals_partei.append(df_deals_fraktion, verify_integrity=True, ignore_index=True, sort=True)
df_deals = df_deals_appended.sort_values(by=['Geschaeftsjahr', 'Geschaeftsnummer'], ascending=False)
df_deals
#result = df1.append(df4, sort=False)

,GeschaeftId,Geschaeftsart,Geschaeftsjahr,Geschaeftsnummer,Geschaeftstitel
0,12f72f93-eef1-4dea-af7c-40dc4f7201c8,Postulat,2021,312,Realisierung eines durchgängigen (Floh-)Markts...
608,a23937ff-a9af-4a6d-8d28-e37cd86dd198,Postulat,2021,308,Auflösung des Bundesasylzentrums Duttweiler un...
1,23e9191b-f4d1-4897-8380-074d3918f98e,Postulat,2021,302,Gesamtenergiebilanz für städtische Gebäude bei...
609,5f1f5506-a655-4a27-8ce7-5e0953b205bc,Erklärung,2021,300,Kommunaler Richtplan Verkehr der Stadt Zürich
2,513e2c8e-60ae-42af-a571-0c7a031922a6,Schriftliche Anfrage,2021,291,"Aktualisierung der Schulwegkarte, Kadenz der A..."
...,...,...,...,...,...
603,1afa9df6-8635-480d-a8b9-b8214b7c6613,Motion,1997,285,"Behördenmitglieder, Neuregelung der Entschädig..."
604,c928d093-0db6-4eaa-b30c-6196035b99e5,Postulat,1996,490,"Zoo Erweiterung, Verlängerung der Tramlinie 6 ..."
605,055190a3-536d-4dbe-97fb-a39677fc57d9,Postulat,1996,100,"Pumpspeicherwerk Grimsel West, Verhinderung we..."
606,09ece0e7-4efc-4fcc-a825-847157382b42,Postulat,1994,424,"Schulbehörden, Neufestlegung der Strukturen"


In [158]:
df_deals = df_deals_appended.sort_values(by=['Geschaeftsjahr', 'Geschaeftsnummer'], ascending=False)


Check Output des Dataframes

In [159]:
df_deals.head()

,GeschaeftId,Geschaeftsart,Geschaeftsjahr,Geschaeftsnummer,Geschaeftstitel
0,12f72f93-eef1-4dea-af7c-40dc4f7201c8,Postulat,2021,312,Realisierung eines durchgängigen (Floh-)Markts...
608,a23937ff-a9af-4a6d-8d28-e37cd86dd198,Postulat,2021,308,Auflösung des Bundesasylzentrums Duttweiler un...
1,23e9191b-f4d1-4897-8380-074d3918f98e,Postulat,2021,302,Gesamtenergiebilanz für städtische Gebäude bei...
609,5f1f5506-a655-4a27-8ce7-5e0953b205bc,Erklärung,2021,300,Kommunaler Richtplan Verkehr der Stadt Zürich
2,513e2c8e-60ae-42af-a571-0c7a031922a6,Schriftliche Anfrage,2021,291,"Aktualisierung der Schulwegkarte, Kadenz der A..."


Exportiere die Geschäfte dieser Anfrage als Excel und als CSV.


In [116]:
#df_deals.to_csv('deals.csv')  
#df_deals.to_excel('deals.xlsx')  

## Details der Gemeinderats-Geschäfte abfragen

Die Abfrage über den Geschäftsendpoint ('https://www.gemeinderat-zuerich.ch/api/Geschaeft') liefert keine detaillierten Daten. Z.B wer das Geschäft eingereicht hat oder wie der aktuelle Stand dabei ist.

Dazu muss der Endpoint zu den Geschäftsdetails ('https://www.gemeinderat-zuerich.ch/api/Geschaeft/details') abgefragt werden. Dieser benötigt als Input die ID des Geschäfts (die gid).


In [160]:
def get_geschaefte_details(index):
    #Herauslesen der GeschaeftId aus dem Dataframe df_deals
    query = {'gid': df_deals['GeschaeftId'][index]}
    #print(query)
    headers = {'Accept': 'application/json'}
    r = requests.get('https://www.gemeinderat-zuerich.ch/api/Geschaeft/details', params=query, headers=headers, verify=SSL_VERIFY) 
    details = r.json()
    # Da komischerweise in den Details die gid nicht mehr vorkommt, weise ich sie hier explizit zu, damit sie im Output vorkommt.
    details['gid'] = df_deals['GeschaeftId'][index]
    return details
 
#Iterrows ist nicht die schnellste Variante der Abfrage. Aber hier dennoch zweckmässig.
deals_details = [get_geschaefte_details(index) for index, row in df_deals.iterrows()]
print("done")




done


Einlsesen des JSON-Outputs in einen DataFrame.
Beachte die Struktur des Outputs. Siehe auch https://opendatazurich.github.io/ris-api/docs/#/Geschaeft/get_Geschaeft_details > Model

In [161]:
df_deals_details = pd.DataFrame(deals_details)
df_deals_details



,GeschaeftTitel,GeschaeftsJahr,GeschaeftsNummer,GeschaeftsArt,Erstunterzeichnender,ZweiterErstunterzeichnender,EinreichendeGruppe,PendentBei,FristBis,Departemente,Ablaufschritte,VerknuepfteGeschaefte,gid
0,Realisierung eines durchgängigen (Floh-)Markts...,2021,312,Postulat,"{'Id': '4a1be35e-3f69-48a4-85b1-3fee07b006ae',...","{'Id': 'ad19c2cb-10d3-426a-a915-45763615ca0c',...",[],Stadtrat,2021-10-07T00:00:00,"[{'Id': 1, 'Name': 'Zuteilung ausstehend'}]","[{'Datum': '2021-07-07T00:00:00', 'Created': '...",[],12f72f93-eef1-4dea-af7c-40dc4f7201c8
1,Auflösung des Bundesasylzentrums Duttweiler un...,2021,308,Postulat,None,None,"[{'Name': 'AL', 'Id': '53bf7290-4055-455b-bb1f...",Stadtrat,2021-10-07T00:00:00,"[{'Id': 1, 'Name': 'Zuteilung ausstehend'}]","[{'Datum': '2021-07-07T00:00:00', 'Created': '...",[{'GeschaeftId': '51289cf9-84ac-481f-8b74-edae...,a23937ff-a9af-4a6d-8d28-e37cd86dd198
2,Gesamtenergiebilanz für städtische Gebäude bei...,2021,302,Postulat,"{'Id': '9e46c2e0-626a-4d19-82e8-de1b2a3e216d',...","{'Id': 'bbfb8ca0-91b1-448d-b1fc-c5e3bb35c158',...",[],Gemeinderat,None,"[{'Id': 8, 'Name': 'Hochbaudepartement (HBD)'}]","[{'Datum': '2021-07-07T00:00:00', 'Created': '...",[],23e9191b-f4d1-4897-8380-074d3918f98e
3,Kommunaler Richtplan Verkehr der Stadt Zürich,2021,300,Erklärung,None,None,"[{'Name': 'AL', 'Id': '53bf7290-4055-455b-bb1f...",,0001-01-01T00:00:00,[],"[{'Datum': '2021-06-30T00:00:00', 'Created': '...",[{'GeschaeftId': '5b9b4c53-433a-4aed-af3b-bfed...,5f1f5506-a655-4a27-8ce7-5e0953b205bc
4,"Aktualisierung der Schulwegkarte, Kadenz der A...",2021,291,Schriftliche Anfrage,"{'Id': 'ad19c2cb-10d3-426a-a915-45763615ca0c',...","{'Id': 'bbfb8ca0-91b1-448d-b1fc-c5e3bb35c158',...",[],Stadtrat,2021-09-23T21:43:24.577,"[{'Id': 13, 'Name': 'Sicherheitsdepartement (S...","[{'Datum': '2021-06-23T00:00:00', 'Created': '...",[],513e2c8e-60ae-42af-a571-0c7a031922a6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,"Behördenmitglieder, Neuregelung der Entschädig...",1997,285,Motion,"{'Id': 'c05a85e4-caf9-48dd-9959-ea4707c84065',...","{'Id': 'b37fdd87-03a3-4505-9b39-85823968e10d',...",[],,0001-01-01T00:00:00,"[{'Id': 4, 'Name': 'Finanzdepartement (FD)'}]","[{'Datum': '1998-05-13T00:00:00', 'Created': '...",[],1afa9df6-8635-480d-a8b9-b8214b7c6613
957,"Zoo Erweiterung, Verlängerung der Tramlinie 6 ...",1996,490,Postulat,"{'Id': 'ee86db77-a1a9-4def-8b78-b5e4ecd27c9a',...",None,[],,0001-01-01T00:00:00,"[{'Id': 9, 'Name': 'Departement der Industriel...","[{'Datum': '2009-09-17T00:00:00', 'Created': '...",[],c928d093-0db6-4eaa-b30c-6196035b99e5
958,"Pumpspeicherwerk Grimsel West, Verhinderung we...",1996,100,Postulat,"{'Id': '3b7ede89-f4fd-444b-81a7-53a66560333e',...","{'Id': 'ee86db77-a1a9-4def-8b78-b5e4ecd27c9a',...",[],Stadtrat,None,"[{'Id': 9, 'Name': 'Departement der Industriel...","[{'Datum': '2020-11-25T00:00:00', 'Created': '...",[{'GeschaeftId': 'ef0539fa-2bb4-45a2-9c42-81c2...,055190a3-536d-4dbe-97fb-a39677fc57d9
959,"Schulbehörden, Neufestlegung der Strukturen",1994,424,Postulat,"{'Id': 'e1297508-be1a-4c61-81b6-a8f501114378',...",None,[],,0001-01-01T00:00:00,"[{'Id': 10, 'Name': 'Schul- und Sportdeparteme...","[{'Datum': '2005-03-02T00:00:00', 'Created': '...",[{'GeschaeftId': 'dacb1cc2-3170-4291-baaa-fe99...,09ece0e7-4efc-4fcc-a825-847157382b42


In der Anfrage ist ein Link auf das Geschäft gewünscht. Die URL hat einen fixen Teil an den am Ende die gid des Geschäfts angehängt wird.
Da der Link nicht im Datenmodell vorhanden ist, wird er hier erstellt und dem DataFrame angehängt.


In [162]:
#df_deals_details['joined_col'] = df_deals_details.GeschaeftsArt.str.cat(df_deals_details.PendentBei)
basis_url_geschaefte = "https://www.gemeinderat-zuerich.ch/geschaefte/detailansicht-geschaeft?gid="

df_deals_details['deals_details_url'] = basis_url_geschaefte + df_deals_details.gid

#df_deals_details.head(3)
#print(basis_url_geschaefte +''+ df_deals_details.gid)


Exportiere den erstellten DataFrame.
Z.K. im DataFrame sieht es aus als ob die URL nicht sauber Concatenated wird. Im Excel-Output ist jedoch alles i.O.

In [163]:
df_deals_details.to_excel('deals_details.xlsx')  
#url_geschaefte = "https://www.gemeinderat-zuerich.ch/geschaefte/detailansicht-geschaeft?gId="df_deals_details['gid']

Z.K. hier eine kompliziertere Version wie die URL konstruiert werden kann. Ist aber letztlich nicht nötig mit Pandas...

In [ ]:

"""
for index, row in df_deals_details.iterrows():

    url_geschaefte = "https://www.gemeinderat-zuerich.ch/geschaefte/detailansicht-geschaeft?gid="
    #gid=deals_details['gid'][index]
    a=df_deals_details['gid'][index]

    print(url_geschaefte+''+a)
"""

'\nfor index, row in df_deals_details.iterrows():\n\n    url_geschaefte = "https://www.gemeinderat-zuerich.ch/geschaefte/detailansicht-geschaeft?gid="\n    #gid=deals_details[\'gid\'][index]\n    a=df_deals_details[\'gid\'][index]\n\n    print(url_geschaefte+\'\'+a)\n'

In [164]:
df_deals_details.columns


Index(['GeschaeftTitel', 'GeschaeftsJahr', 'GeschaeftsNummer', 'GeschaeftsArt',
       'Erstunterzeichnender', 'ZweiterErstunterzeichnender',
       'EinreichendeGruppe', 'PendentBei', 'FristBis', 'Departemente',
       'Ablaufschritte', 'VerknuepfteGeschaefte', 'gid', 'deals_details_url'],
      dtype='object')

### Normalisierung der Geschäftsdetails

Der Output der Detailinformationen ist verschachtelt, wie im [Datenmodell](https://opendatazurich.github.io/ris-api/docs/#/Geschaeft/get_Geschaeft_details) beschrieben. In den nächsten Schritten wird der DataFrame mit den Datails zu den Geschäften normalisiert. Siehe dazu z.B folgende Links:
- https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.io.json.json_normalize.html
- https://towardsdatascience.com/all-pandas-json-normalize-you-should-know-for-flattening-json-13eae1dfb7dd
- https://blog.softhints.com/python-normalize-json-pandas/


Hier im ersten Versuch werden keine weiteren Parameter angegeben. Es ist jedoch ersichtlich, dass dies nicht dem von uns erhofften Output entspricht. Es ist unklar, was mit den Erstunterzeichnenden geschehen ist ... jedenfalls nützt das so nichts.

In [166]:
from pandas.io.json import json_normalize

In [167]:

#df_deals_details_normalized = pd.json_normalize(deals_details, sep="_")

df_deals_details_normalized = pd.json_normalize(deals_details)
#, max_level=1


df_deals_details_normalized.columns
#df_deals_details_normalized.head(3)
#df_deals_details_normalized.to_excel(path+'deals_details_normalized.xlsx')  

Index(['GeschaeftTitel', 'GeschaeftsJahr', 'GeschaeftsNummer', 'GeschaeftsArt',
       'EinreichendeGruppe', 'PendentBei', 'FristBis', 'Departemente',
       'Ablaufschritte', 'VerknuepfteGeschaefte', 'gid',
       'Erstunterzeichnender.Id', 'Erstunterzeichnender.Name',
       'Erstunterzeichnender.Vorname', 'Erstunterzeichnender.Titel',
       'Erstunterzeichnender.Partei', 'Erstunterzeichnender.Wahlkreis',
       'Erstunterzeichnender.WahlkreisOrderBy',
       'ZweiterErstunterzeichnender.Id', 'ZweiterErstunterzeichnender.Name',
       'ZweiterErstunterzeichnender.Vorname',
       'ZweiterErstunterzeichnender.Titel',
       'ZweiterErstunterzeichnender.Partei',
       'ZweiterErstunterzeichnender.Wahlkreis',
       'ZweiterErstunterzeichnender.WahlkreisOrderBy', 'Erstunterzeichnender',
       'ZweiterErstunterzeichnender'],
      dtype='object')

In [168]:
url_geschaefte = "https://www.gemeinderat-zuerich.ch/geschaefte/detailansicht-geschaeft"
df_deals_details_normalized['deals_details_url'] = url_geschaefte +'?gid='+ df_deals_details_normalized.gid

df_deals_details_normalized.head(2)
df_deals_details_normalized.columns


Index(['GeschaeftTitel', 'GeschaeftsJahr', 'GeschaeftsNummer', 'GeschaeftsArt',
       'EinreichendeGruppe', 'PendentBei', 'FristBis', 'Departemente',
       'Ablaufschritte', 'VerknuepfteGeschaefte', 'gid',
       'Erstunterzeichnender.Id', 'Erstunterzeichnender.Name',
       'Erstunterzeichnender.Vorname', 'Erstunterzeichnender.Titel',
       'Erstunterzeichnender.Partei', 'Erstunterzeichnender.Wahlkreis',
       'Erstunterzeichnender.WahlkreisOrderBy',
       'ZweiterErstunterzeichnender.Id', 'ZweiterErstunterzeichnender.Name',
       'ZweiterErstunterzeichnender.Vorname',
       'ZweiterErstunterzeichnender.Titel',
       'ZweiterErstunterzeichnender.Partei',
       'ZweiterErstunterzeichnender.Wahlkreis',
       'ZweiterErstunterzeichnender.WahlkreisOrderBy', 'Erstunterzeichnender',
       'ZweiterErstunterzeichnender', 'deals_details_url'],
      dtype='object')

In [169]:
df_deals_details_normalized.to_excel('deals_details_normalized.xlsx')  


In [ ]:
#deals_details

So können z.B. die Ablaufschritte einzeln noch herausgelesen werden 

In [ ]:
df_deals_details_normalized2 = pd.json_normalize(
    deals_details, 
    record_path=['Ablaufschritte'], 
    meta=[['Id', 'Name', 'Vorname']],
    meta_prefix='meta-',
    record_prefix='path-',
    errors='ignore' 
)



In [ ]:
#deals_details
df_deals_details_normalized2.head(2)
#df_deals_details_normalized2.columns

,path-Datum,path-Created,path-Titel,path-Protokolleintrag,path-AblaufschrittDokumente,path-BeschlussNrGR,path-SitzungsNr,path-AudioLink,path-AudioLinkText,meta-Id.Name.Vorname
0,2021-07-07T00:00:00,2021-07-08T11:07:54.98,"Eingang, Frist 3 Monate","<p style=""margin: 0px 0px 0px 0px;text-align: ...","[{'Dokumenttitel': '2021_0312.pdf', 'Id': '5d0...",4196.0,157.0,None,None,NaN
1,2021-07-07T00:00:00,2021-07-07T13:27:48.893,"Stadtrat, Entgegennahme",None,[],NaN,NaN,None,None,NaN
